In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from feature import featureExtractionPipeline
from scipy.stats import kurtosis, skew
from tqdm import tqdm_notebook as tqdm
from pyprep.noisy import Noisydata
from multiprocessing import Pool 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import mne
import os

In [2]:
def getMneRaw(data_to_process):
    sfreq = 128
    channel_names = ['F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3',
                     'P4', 'FC5', 'FC1', 'FC2', 'FC4', 'CP5',
                     'CP1', 'CP2', 'CP4']
    channel_type = {k:'eeg' for k in channel_names}
    info = mne.create_info(channel_names, sfreq,verbose=False)
    raw = mne.io.RawArray(data_to_process, info, verbose=False)
    raw.set_channel_types(channel_type)
    montage = mne.channels.read_montage('standard_1020')
    raw.set_montage(montage,verbose=False)
    raw.filter(0,50,fir_design='firwin',verbose=False)
    return raw

def getFileInfo(file_path):
    try:
        raw_data = pd.read_csv(file_path,header=None)
        data_to_process = raw_data[:-1]
        label = int( list(set(raw_data.iloc[len(raw_data) -1 ].values))[0] )
        raw = getMneRaw(data_to_process)
        return [ raw, label ]
    except:
        return [0, 0]
    
def getChannelFeatures(raw_data_with_label):
    try:
        raw_data = raw_data_with_label[0]
        label = raw_data_with_label[1]
        channels = raw_data.get_data()
        features = []
        for channel in channels:
            feature = featureExtractionPipeline(channel).runPipeline()
            feature = sum(feature, [])
            features.append( feature )
        feature_df = pd.DataFrame( features )
        feature_df = feature_df.drop([15,16], axis=1)
        feature_df['label'] = label
        return feature_df
    except:
        pass
    
def getAlphaBeta(raw_data):
    alpha = raw_data.copy()
    beta = raw_data.copy()
    alpha = alpha.filter(8,13)
    beta = beta.filter(13,30)
    return [alpha, beta]

In [3]:
files = glob.glob('../data/2colourswifi_transition/*.csv')

In [4]:
raw_data_list = []
for i in tqdm(files):
    raw_data_label = getFileInfo(i)
    raw_data_list.append( getAlphaBeta( raw_data_label[0] ) + [raw_data_label[1]] )
    break

In [5]:
temp = raw_data_list[0][0].get_data()[0]

In [7]:
featureExtractionPipeline(temp[:1000]).runPipeline()

[-1.6590953190410335e-07,
 1.7378744302480348e-05,
 -1.7155400316652635e-15,
 5.95462694912169e-19,
 0.5485269646340741,
 3.4847470763356266e-06,
 6.969494152671253e-09,
 -2.8746348456023505e-05,
 2.9231145349918257e-05,
 4.791540409262503e-06,
 1.4740221725525295e-17,
 5.900979013562386e-21,
 1.1479454033722118e-08,
 0.012197523107014694,
 38.85704452745294,
 0.00030204827950377003,
 0.038278958815092044,
 0.14694565409378293,
 0.16418662307500695,
 0.10447438544090203,
 0.01549886790111525]

In [8]:
temp.shape

(14023,)

In [ ]:
'''
feature_array = []
for i in tqdm(raw_data_list):
    feature_array.append( getChannelFeatures(i) )
'''